# CUSTOMER LIFETIME VALUE PREDICTION

In [26]:
import datetime as dt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from datetime import timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

df_ = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")

In [27]:
# Outlier Handling Functions
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [28]:
# Data Preperation
df = df_.copy()
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")
df["TotalPrice"] = df["Quantity"] * df["Price"]

df_uk = df[df["Country"] == "United Kingdom"]
last_date = df["InvoiceDate"].max()
today_date = last_date + timedelta(days=3)
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.00000,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom,15.30000
1,536365,71053,WHITE METAL LANTERN,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.00000,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom,22.00000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000


In [29]:
# RFM Table
rfm_from_uk = df_uk.groupby("Customer ID").agg({"InvoiceDate": [lambda date: (date.max() - date.min()).days, #Recency
                                                                lambda date: (today_date - date.min()).days], #Tenure
                                                "Invoice": lambda num: num.nunique(), #Frequency
                                                "TotalPrice": lambda TotalPrice: TotalPrice.sum()}) #Monetary
rfm_from_uk.columns = rfm_from_uk.columns.droplevel(0)
rfm_from_uk.columns = ['Recency_cltv_p', 'T', 'Frequency', 'Monetary']
rfm_from_uk["Monetary"] = rfm_from_uk["Monetary"] / rfm_from_uk["Frequency"]
rfm_from_uk.rename(columns={"Monetary": "Monetary_avg"}, inplace=True)
rfm_from_uk.index = rfm_from_uk.index.astype(int)
rfm_from_uk.head()

,Recency_cltv_p,T,Frequency,Monetary_avg
Customer ID,,,,
12346,0,328,1,310.44000
12747,366,371,11,381.45545
12748,372,376,210,154.19243
12749,209,215,5,815.58800
12820,323,329,4,235.58500


In [30]:
rfm_from_uk["Recency_weekly_p"] = rfm_from_uk["Recency_cltv_p"] / 7
rfm_from_uk["T_weekly"] = rfm_from_uk["T"] / 7
rfm_from_uk = rfm_from_uk[(rfm_from_uk["Monetary_avg"] > 0)]
rfm_from_uk = rfm_from_uk[(rfm_from_uk['Frequency'] > 1)]
rfm_from_uk["Frequency"] = rfm_from_uk["Frequency"].astype(int)
rfm_fr_uk_cltv = rfm_from_uk.copy()
rfm_fr_uk_cltv.head()

,Recency_cltv_p,T,Frequency,Monetary_avg,Recency_weekly_p,T_weekly
Customer ID,,,,,,
12747,366,371,11,381.45545,52.28571,53.00000
12748,372,376,210,154.19243,53.14286,53.71429
12749,209,215,5,815.58800,29.85714,30.71429
12820,323,329,4,235.58500,46.14286,47.00000
12822,16,89,2,474.44000,2.28571,12.71429


In [31]:
# BGNBD Model
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(rfm_fr_uk_cltv["Frequency"],
        rfm_fr_uk_cltv["Recency_weekly_p"],
        rfm_fr_uk_cltv["T_weekly"])

<lifetimes.BetaGeoFitter: fitted with 2570 subjects, a: 0.12, alpha: 11.76, b: 2.54, r: 2.22>

In [32]:
# Gamma Gamma Model
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(rfm_fr_uk_cltv["Frequency"], rfm_fr_uk_cltv["Monetary_avg"])

<lifetimes.GammaGammaFitter: fitted with 2570 subjects, p: 3.81, q: 0.35, v: 3.75>

In [33]:
# Prediction of CLTW With BG-NBD and GammaGamma Model for 6 moths
cltv_fr_uk_6 = ggf.customer_lifetime_value(bgf,
                                        rfm_fr_uk_cltv["Frequency"],
                                        rfm_fr_uk_cltv["Recency_weekly_p"],
                                        rfm_fr_uk_cltv["T_weekly"],
                                        rfm_fr_uk_cltv["Monetary_avg"],
                                        freq="W",
                                        time=6,
                                        discount_rate=0.01)
cltv_fr_uk_6 = cltv_fr_uk_6.reset_index()
cltv_fr_uk_6.sort_values(by="clv", ascending=False).head(20)

,Customer ID,clv
2486,18102,91772.70885
589,14096,54724.23797
2184,17450,52408.80352
2213,17511,36679.49567
1804,16684,26017.11669
406,13694,25033.52471
587,14088,24908.04898
1173,15311,23424.92495
1485,16000,23132.49228
133,13089,22782.70836


In [34]:
rfm_fr_uk_cltv.index.isin(cltv_fr_uk_6["Customer ID"]).all()
rfm_fr_uk_cltv = rfm_fr_uk_cltv.merge(cltv_fr_uk_6, how="left", on="Customer ID")
rfm_fr_uk_cltv.head()

,Customer ID,Recency_cltv_p,T,Frequency,Monetary_avg,Recency_weekly_p,T_weekly,clv
0,12747,366,371,11,381.45545,52.28571,53.00000,1928.20922
1,12748,372,376,210,154.19243,53.14286,53.71429,12286.30482
2,12749,209,215,5,815.58800,29.85714,30.71429,3427.30100
3,12820,323,329,4,235.58500,46.14286,47.00000,627.23813
4,12822,16,89,2,474.44000,2.28571,12.71429,1578.06584


In [35]:
scaler = MinMaxScaler(feature_range=(1, 100))
scaler.fit(rfm_fr_uk_cltv[["clv"]])
rfm_fr_uk_cltv["cltv_scaled"] = scaler.transform(rfm_fr_uk_cltv[["clv"]])
rfm_fr_uk_cltv["cltv_p_segment"] = pd.qcut(rfm_fr_uk_cltv["cltv_scaled"], 3, labels=["C", "B", "A"])
rfm_fr_uk_cltv.head()

,Customer ID,Recency_cltv_p,T,Frequency,Monetary_avg,Recency_weekly_p,T_weekly,clv,cltv_scaled,cltv_p_segment
0,12747,366,371,11,381.45545,52.28571,53.00000,1928.20922,3.08006,A
1,12748,372,376,210,154.19243,53.14286,53.71429,12286.30482,14.25388,A
2,12749,209,215,5,815.58800,29.85714,30.71429,3427.30100,4.69721,A
3,12820,323,329,4,235.58500,46.14286,47.00000,627.23813,1.67663,B
4,12822,16,89,2,474.44000,2.28571,12.71429,1578.06584,2.70234,A


In [36]:
rfm_fr_uk_cltv.groupby("cltv_p_segment").agg({"Recency_weekly_p": ["median"],
                                              "T_weekly": ["median"],
                                              "Frequency": ["median"],
                                              "Monetary_avg": ["median"],
                                              "cltv_scaled": ["median"]})

,Recency_weekly_p,T_weekly,Frequency,Monetary_avg,cltv_scaled
,median,median,median,median,median
cltv_p_segment,,,,,
C,23.14286,41.71429,3,167.03500,1.38012
B,32.57143,40.42857,4,290.69694,2.01818
A,34.28571,39.57143,6,448.72250,3.29295


In [37]:
xy = rfm_fr_uk_cltv.groupby("Customer ID").agg({"clv": "mean"}).sort_values(by="clv", ascending=False)
xy = xy.reset_index()
xy["Ratio"] = (xy["clv"] / xy["clv"].sum()) * 100
xy["Cum_Ratio"] = xy["Ratio"].cumsum()
xy.head()

,Customer ID,clv,Ratio,Cum_Ratio
0,18102,91772.70885,2.36421,2.36421
1,14096,54724.23797,1.40978,3.77399
2,17450,52408.80352,1.35013,5.12412
3,17511,36679.49567,0.94492,6.06904
4,16684,26017.11669,0.67024,6.73928


In [38]:
xy["Top_Flag"] = xy["Cum_Ratio"]
xy["Top_Flag"] = ["top_flag 1" if i <= 80 else "top_flag 0" for i in xy["Top_Flag"]]
xy = xy.drop(["clv", "Ratio", "Cum_Ratio"], axis=1)
rfm_fr_uk_cltv = rfm_fr_uk_cltv.merge(a, how="left", on="Customer ID")
rfm_fr_uk_cltv.head()
len(rfm_fr_uk_cltv[rfm_fr_uk_cltv["Top_Flag"] == "top_flag 1"]) / len(rfm_fr_uk_cltv) * 100
# Pareto: 44.04

44.04669260700389